<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploring-the-existing-data-tables" data-toc-modified-id="Exploring-the-existing-data-tables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploring the existing data tables</a></span><ul class="toc-item"><li><span><a href="#orderdetails-table" data-toc-modified-id="orderdetails-table-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><code>orderdetails</code> table</a></span></li><li><span><a href="#products-table" data-toc-modified-id="products-table-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span><code>products</code> table</a></span></li></ul></li><li><span><a href="#Joins-on-data-tables" data-toc-modified-id="Joins-on-data-tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Joins on data tables</a></span><ul class="toc-item"><li><span><a href="#Inner-Join-on-products-&amp;-orderdetails" data-toc-modified-id="Inner-Join-on-products-&amp;-orderdetails-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Inner Join on <code>products</code> &amp; <code>orderdetails</code></a></span></li><li><span><a href="#Left-Join-on-products-&amp;-orderdetails" data-toc-modified-id="Left-Join-on-products-&amp;-orderdetails-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Left Join on <code>products</code> &amp; <code>orderdetails</code></a></span><ul class="toc-item"><li><span><a href="#What-if-there-are-no-orders-for-that-product?" data-toc-modified-id="What-if-there-are-no-orders-for-that-product?-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>What if there are no orders for that product?</a></span></li></ul></li><li><span><a href="#Doing-an-aggregations" data-toc-modified-id="Doing-an-aggregations-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Doing an aggregations</a></span><ul class="toc-item"><li><span><a href="#With-Pandas" data-toc-modified-id="With-Pandas-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>With Pandas</a></span></li><li><span><a href="#Same-aggregation-with-SQL" data-toc-modified-id="Same-aggregation-with-SQL-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Same aggregation with SQL</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()

# Exploring the existing data tables

Helper functions so I can use DataFrames

In [ ]:
def sql_with_cols(query,cursor=cur):
  '''
  Gives me the full result (with columns)
  '''
  result = cursor.execute(query).fetchall()
  cols = tuple([description[0] for description in cur.description])
  
  full_result = (cols, result[:])
  return full_result

def df_sql(query, cursor=cur):
    cols, result_data = sql_with_cols(query, cursor)
    return pd.DataFrame(data=result_data, columns=cols)

## `orderdetails` table

In [ ]:
query = """
select 
    * 
from 
    orderdetails
"""

df_orderdetails = df_sql(query)
df_orderdetails.head()

In [ ]:
df_orderdetails.describe()

## `products` table

In [ ]:
query = """
select 
    * 
from 
    products
"""

df_products = df_sql(query)

In [ ]:
df_products.head()

In [ ]:
df_products.describe()

# Joins on data tables

## Inner Join on `products` & `orderdetails`

In [ ]:
query = """
select * 
from 
    orderdetails
    join products
        on orderdetails.productCode = products.productCode
"""
df_join_products_orderdetails = df_sql(query)

In [ ]:
# Rename the `productCode` columns since they are repeated
old_cols = list(df_join_products_orderdetails.columns)
new_cols = (
    [f'o.{c}' for c in old_cols[:5]]
    +
    [f'p.{c}' for c in old_cols[5:]]
)
df_join_products_orderdetails.columns = new_cols

In [ ]:
df_join_products_orderdetails.head()

In [ ]:
df_join_products_orderdetails.describe()

## Left Join on `products` & `orderdetails`

For all the products we have, get the order information

In [ ]:
query = """
select 
    * 
from 
    products as p
    left join orderdetails as o
        on o.productCode = p.productCode
"""

df_innerjoin_products_orderdetails = df_sql(query)

In [ ]:
# Rename the `productCode` columns since they are repeated
old_cols = list(df_innerjoin_products_orderdetails.columns)
new_cols = (
    [f'p.{c}' for c in old_cols[:9]]
    +
    [f'o.{c}' for c in old_cols[9:]]
)
df_innerjoin_products_orderdetails.columns = new_cols

In [ ]:
df_innerjoin_products_orderdetails.head()

In [ ]:
df_innerjoin_products_orderdetails.describe()

### What if there are no orders for that product?

In [ ]:
df_innerjoin_products_orderdetails[df_innerjoin_products_orderdetails['o.orderNumber'].isnull()]

In [ ]:
# Does that product show up in the last join?
mask = df_join_products_orderdetails['p.productCode'] == 'S18_3233'
print(mask.sum())

In [ ]:
df_join_products_orderdetails[mask]

## Doing an aggregations

### With Pandas

In [ ]:
(
df_join_products_orderdetails
    .groupby(['p.productCode'])['p.productName']
    .count()
    .sort_values()
)

### Same aggregation with SQL

In [ ]:
query = """
select
    p.productCode as 'p.productCode',
    count(*) as product_count
from 
    orderdetails as o
    join products as p
        on o.productCode = p.productCode
group by
    p.productCode
order by
    product_count
"""
df = df_sql(query)